In [1]:
import json
import pandas as pd
import numpy as np
from os import path as path
import glob
import imageio
import numpy.ma as ma
import pathlib


In [15]:
A = imageio.imread('stimuli/A.png')[:,:,1]
S = imageio.imread('stimuli/S.png')[:,:,1]

targets = {'A': A, 'S':S}

def get_correlation_with_S(row):
    if str(row['presented_pixel_data'])=='nan':
        return 0
    else:
        presented_pixel_data = np.array(row['presented_pixel_data'], dtype=float)
        correlation_per_frame = np.array([ma.corrcoef(ma.masked_invalid(frame.flatten()),
                                            ma.masked_invalid(S.flatten()))[0,1] for frame in presented_pixel_data])
        return correlation_per_frame
    
def get_correlation_with_A(row):
    if str(row['presented_pixel_data'])=='nan':
        return 0
    else:
        presented_pixel_data = np.array(row['presented_pixel_data'], dtype=float)
        correlation_per_frame = np.array([ma.corrcoef(ma.masked_invalid(frame.flatten()),
                                            ma.masked_invalid(A.flatten()))[0,1] for frame in presented_pixel_data])
        return correlation_per_frame
    
def get_correlation_with_target_letter(row):
    if row.target=='S':
        return get_correlation_with_S(row)
    elif row.target=='A':
        return(get_correlation_with_A(row))
    
def get_correlation_with_other_letter(row):
    if row.target=='A':
        return get_correlation_with_S(row)
    elif row.target=='S':
        return(get_correlation_with_A(row))
    
def get_correlations(row):
    target = get_correlation_with_target_letter(row) 
    other = get_correlation_with_other_letter(row)   
    
    if target is None:
        return [[0],[0]]
        
    return list(zip(list(target), list(other)))


def get_SSE(vector1,vector2):
    return sum((a - b)**2 for a, b in zip(vector1, vector2) if not (np.isnan(a) or np.isnan(b)))

def get_SSE_with_S(row):
    if str(row['presented_pixel_data'])=='nan':
        return 0
    else:
        presented_pixel_data = np.array(row['presented_pixel_data'], dtype=float)
        SSE_per_frame = np.array([get_SSE(frame.flatten(),S.flatten()) for frame in presented_pixel_data])
        return SSE_per_frame
    
def get_SSE_with_A(row):
    if str(row['presented_pixel_data'])=='nan':
        return 0
    else:
        presented_pixel_data = np.array(row['presented_pixel_data'], dtype=float)
        SSE_per_frame = np.array([get_SSE(frame.flatten(),A.flatten()) for frame in presented_pixel_data])
        return SSE_per_frame
    
def get_SSE_with_target_letter(row):
    if row.target=='S':
        return get_SSE_with_S(row)
    elif row.target=='A':
        return(get_SSE_with_A(row))
    
def get_SSE_with_other_letter(row):
    if row.target=='A':
        return get_SSE_with_S(row)
    elif row.target=='S':
        return(get_SSE_with_A(row))
    
def get_correlations_and_SSEs(row):
    cor_target = get_correlation_with_target_letter(row) 
    if cor_target is None:
        return [[0],[0],[0],[0]]
    
    cor_other = get_correlation_with_other_letter(row)   
    SSE_target = get_SSE_with_target_letter(row) 
    SSE_other = get_SSE_with_other_letter(row)   

    return list(zip(list(cor_target), list(cor_other),
                   list(SSE_target), list(SSE_other)))

def get_correlations(row):
    cor_target = get_correlation_with_target_letter(row) 
    if cor_target is None:
        return [[0],[0]]
    
    cor_other = get_correlation_with_other_letter(row)   
    return list(zip(list(cor_target), list(cor_other)))
    
    
def get_correlation_with_mask(row):
    if str(row['presented_pixel_data'])=='nan':
        return 0
    else:
        presented_pixel_data = np.array(row['presented_pixel_data'], dtype=float)
        target = targets[row['target']];
        correlation_with_nans = ma.corrcoef(np.where(np.isnan(presented_pixel_data[0,:,:].flatten()),1,0),
                                            target.flatten())[0,1]
        return correlation_with_nans

In [30]:
def get_num_pixels_hidden(row):
    if str(row['presented_pixel_data'])=='nan':
        return 0
    else:
        presented_pixel_data = row['presented_pixel_data']
        first_frame = presented_pixel_data[0];
        number_of_hidden_pixels = len([item for sublist in first_frame for item in sublist if str(item) == 'None'])
        return number_of_hidden_pixels

def to_csv_multiple_files(dirname):
    dfs=[]
    json_files = glob.glob(path.join(dirname,'*','*.json'))
    for i,file in enumerate(json_files):
                mydf = pd.read_json(file)
                mydf['num_pixels_hidden'] = mydf.apply(get_num_pixels_hidden, axis=1)
                mydf['correlation_with_target_letter'] = mydf.apply(get_correlation_with_target_letter, axis=1) 
                mydf['correlation_with_mask'] = mydf.apply(get_correlation_with_mask, axis=1)
                mydf = mydf.explode('correlation_with_target_letter');
                mydf['frame_index'] = mydf.groupby('trial_index').cumcount()
                dfs.append(mydf.drop('presented_pixel_data',axis=1));
    group_df = pd.concat(dfs)
    group_df.to_csv(dirname+'/all_data.csv', index=False)
    return(group_df)

#12.03.2023: JATOS changed their export data format (the tree structure is more convolved now)
def to_csv_multiple_files_new_format(dirname):
    dfs=[]
    json_files = pathlib.Path(dirname).glob('**/*.json')
    for i,file in enumerate(json_files):
                mydf = pd.read_json(file)
                mydf['num_pixels_hidden'] = mydf.apply(get_num_pixels_hidden, axis=1)
                mydf['correlation_with_target_letter'] = mydf.apply(get_correlation_with_target_letter, axis=1)   
                mydf['correlation_with_mask'] = mydf.apply(get_correlation_with_mask, axis=1)
                mydf = mydf.explode('correlation_with_target_letter');
                mydf['frame_index'] = mydf.groupby('trial_index').cumcount()
                dfs.append(mydf.drop('presented_pixel_data',axis=1));
    group_df = pd.concat(dfs)
    group_df.to_csv(dirname+'/all_data.csv', index=False)
    return(group_df)

def to_csv_multiple_files_new_format_with_other_letter(dirname):
    dfs=[]
    json_files = pathlib.Path(dirname).glob('**/*.json')
    for i,file in enumerate(json_files):
                mydf = pd.read_json(file)
                mydf['num_pixels_hidden'] = mydf.apply(get_num_pixels_hidden, axis=1)
                mydf['correlations'] = mydf.apply(get_correlations, axis=1)   
                mydf['correlation_with_mask'] = mydf.apply(get_correlation_with_mask, axis=1)
                mydf = mydf.explode('correlations');
                mydf[['correlation_with_target_letter', 'correlation_with_other_letter']] = pd.DataFrame(mydf['correlations'].tolist(), index=mydf.index)
                mydf['frame_index'] = mydf.groupby('trial_index').cumcount()
                dfs.append(mydf.drop('presented_pixel_data',axis=1));
    group_df = pd.concat(dfs)
    group_df.to_csv(dirname+'/all_data.csv', index=False)
    return(group_df)

def to_csv_multiple_files_new_format_with_other_letter_and_SSE(dirname):
    dfs=[]
    json_files = pathlib.Path(dirname).glob('**/*.json')
    for i,file in enumerate(json_files):
            print(i)
            mydf = pd.read_json(file)
            mydf['num_pixels_hidden'] = mydf.apply(get_num_pixels_hidden, axis=1)
            mydf['correlations'] = mydf.apply(get_correlations_and_SSEs, axis=1)   
            mydf['correlation_with_mask'] = mydf.apply(get_correlation_with_mask, axis=1)
            mydf = mydf.explode('correlations');
            mydf[['correlation_with_target_letter', 'correlation_with_other_letter', 
                 'SSE_with_target_letter', 'SSE_with_other_letter']] = pd.DataFrame(mydf['correlations'].tolist(), index=mydf.index)
            mydf['frame_index'] = mydf.groupby('trial_index').cumcount()
            dfs.append(mydf.drop('presented_pixel_data',axis=1));
    group_df = pd.concat(dfs)
    group_df.to_csv(dirname+'/all_data_and_SSE.csv', index=False)
    return(group_df)

In [ ]:
group_df=to_csv_multiple_files(path.join('..','experiments','Exp1pixels','version2','data','jatos_resultfiles_batch1'))

In [ ]:
group_df=to_csv_multiple_files_new_format_with_other_letter(path.join('..','experiments','Exp2rows','data','jatos_resultfiles_batch1'))

C:\Users\tanzor\AppData\Local\Temp/ipykernel_37132/1340491763.py:11: UserWarning: Warning: converting a masked element to nan.
  correlation_per_frame = np.array([ma.corrcoef(ma.masked_invalid(frame.flatten()),
C:\Users\tanzor\AppData\Local\Temp/ipykernel_37132/1340491763.py:20: UserWarning: Warning: converting a masked element to nan.
  correlation_per_frame = np.array([ma.corrcoef(ma.masked_invalid(frame.flatten()),


In [31]:
group_df=to_csv_multiple_files_new_format_with_other_letter_and_SSE(path.join('..','experiments','Exp2rows','data','jatos_resultfiles_batch1'))

0


C:\Users\tanzor\AppData\Local\Temp/ipykernel_37132/1340491763.py:11: UserWarning: Warning: converting a masked element to nan.
  correlation_per_frame = np.array([ma.corrcoef(ma.masked_invalid(frame.flatten()),
C:\Users\tanzor\AppData\Local\Temp/ipykernel_37132/1340491763.py:20: UserWarning: Warning: converting a masked element to nan.
  correlation_per_frame = np.array([ma.corrcoef(ma.masked_invalid(frame.flatten()),


1


C:\Users\tanzor\AppData\Local\Temp/ipykernel_37132/1340491763.py:11: UserWarning: Warning: converting a masked element to nan.
  correlation_per_frame = np.array([ma.corrcoef(ma.masked_invalid(frame.flatten()),
C:\Users\tanzor\AppData\Local\Temp/ipykernel_37132/1340491763.py:20: UserWarning: Warning: converting a masked element to nan.
  correlation_per_frame = np.array([ma.corrcoef(ma.masked_invalid(frame.flatten()),


2


...

C:\Users\tanzor\AppData\Local\Temp/ipykernel_37132/1340491763.py:20: UserWarning: Warning: converting a masked element to nan.
  correlation_per_frame = np.array([ma.corrcoef(ma.masked_invalid(frame.flatten()),
C:\Users\tanzor\AppData\Local\Temp/ipykernel_37132/1340491763.py:11: UserWarning: Warning: converting a masked element to nan.
  correlation_per_frame = np.array([ma.corrcoef(ma.masked_invalid(frame.flatten()),


250


C:\Users\tanzor\AppData\Local\Temp/ipykernel_37132/1340491763.py:20: UserWarning: Warning: converting a masked element to nan.
  correlation_per_frame = np.array([ma.corrcoef(ma.masked_invalid(frame.flatten()),
C:\Users\tanzor\AppData\Local\Temp/ipykernel_37132/1340491763.py:11: UserWarning: Warning: converting a masked element to nan.
  correlation_per_frame = np.array([ma.corrcoef(ma.masked_invalid(frame.flatten()),


251


C:\Users\tanzor\AppData\Local\Temp/ipykernel_37132/1340491763.py:20: UserWarning: Warning: converting a masked element to nan.
  correlation_per_frame = np.array([ma.corrcoef(ma.masked_invalid(frame.flatten()),
C:\Users\tanzor\AppData\Local\Temp/ipykernel_37132/1340491763.py:11: UserWarning: Warning: converting a masked element to nan.
  correlation_per_frame = np.array([ma.corrcoef(ma.masked_invalid(frame.flatten()),


In [ ]:
group_df=to_csv_multiple_files_new_format(path.join('..','experiments','Exp2rowsLong','data','json_data'))

In [ ]:
def get_correlation_with_S_3stim(row):
    if str(row['presented_pixel_data'])=='nan':
        return 0
    else:
        presented_pixel_data = np.array(row['presented_pixel_data'], dtype=float)
        correlation_per_frame = np.array([ma.corrcoef(ma.masked_invalid(frame[1].flatten()),
                                            ma.masked_invalid(S.flatten()))[0,1] for frame in presented_pixel_data])
        return correlation_per_frame
    
def get_correlation_with_A_3stim(row):
    if str(row['presented_pixel_data'])=='nan':
        return 0
    else:
        presented_pixel_data = np.array(row['presented_pixel_data'], dtype=float)
        correlation_per_frame = np.array([ma.corrcoef(ma.masked_invalid(frame[1].flatten()),
                                            ma.masked_invalid(A.flatten()))[0,1] for frame in presented_pixel_data])
        return correlation_per_frame
    
def get_correlation_with_target_letter_3stim(row):
    if row.target=='S':
        return get_correlation_with_S_3stim(row)
    elif row.target=='A':
        return(get_correlation_with_A_3stim(row))
    
    

def to_csv_multiple_files_new_3stim(dirname):
    dfs=[]
    json_files = pathlib.Path(dirname).glob('**/*.json')
    for i,file in enumerate(json_files):
                mydf = pd.read_json(file)
                mydf['correlation_with_target_letter'] = mydf.apply(get_correlation_with_target_letter_3stim, axis=1)   
                mydf = mydf.explode('correlation_with_target_letter');
                mydf['frame_index'] = mydf.groupby('trial_index').cumcount()
                dfs.append(mydf.drop('presented_pixel_data',axis=1));
    group_df = pd.concat(dfs)
    group_df.to_csv(dirname+'/all_data.csv', index=False)
    return(group_df)

group_df=to_csv_multiple_files_new_3stim(path.join('..','experiments','Exp3reference','data','jatos_results_files_batch1'))

In [ ]:
group_df